In [ ]:
%cd ../BackEnd

import json
import pandas as pd
from sqlalchemy.orm import Session
from database import SessionLocal
from models.credit import Credit
from models.person import Person
from models.cast_member import CastMember
from models.crew_member import CrewMember

In [26]:
import json
import pandas as pd
import csv
import os

# CSV file names (these are the same files used for movie credits)
PERSONS_CSV = "tv_persons_temp.csv"
CREDITS_CSV = "tv_credits_temp.csv"
CAST_CSV    = "tv_cast_temp.csv"
CREW_CSV    = "tv_crew_temp.csv"

# Utility functions to load existing keys/pairs from a CSV file
def load_existing_keys(filename, key_index=0):
    keys = set()
    if os.path.exists(filename):
        with open(filename, "r", newline="", encoding="utf-8") as f:
            reader = csv.reader(f)
            next(reader, None)  # skip header
            for row in reader:
                if row and len(row) > key_index:
                    keys.add(row[key_index])
    return keys

def load_existing_pairs(filename, key_index1=0, key_index2=1):
    pairs = set()
    if os.path.exists(filename):
        with open(filename, "r", newline="", encoding="utf-8") as f:
            reader = csv.reader(f)
            next(reader, None)  # skip header
            for row in reader:
                if row and len(row) > key_index2:
                    pairs.add((row[key_index1], row[key_index2]))
    return pairs

# Load global caches from existing CSV files (if any)
global_persons_cache = load_existing_keys(PERSONS_CSV, key_index=0)
global_credits_cache = load_existing_keys(CREDITS_CSV, key_index=0)
global_cast_cache    = load_existing_pairs(CAST_CSV, key_index1=0, key_index2=1)  # (credit_id, person_id)
global_crew_cache    = load_existing_pairs(CREW_CSV, key_index1=0, key_index2=1)  # (credit_id, person_id)

print(f"Loaded {len(global_persons_cache)} persons, {len(global_credits_cache)} credits, "
      f"{len(global_cast_cache)} cast entries, and {len(global_crew_cache)} crew entries from existing CSVs.")

# Open CSV files in append mode (so that new TV data gets added after movie data)
persons_file = open(PERSONS_CSV, "w", newline="", encoding="utf-8")
credits_file = open(CREDITS_CSV, "w", newline="", encoding="utf-8")
cast_file    = open(CAST_CSV, "w", newline="", encoding="utf-8")
crew_file    = open(CREW_CSV, "w", newline="", encoding="utf-8")

persons_writer = csv.writer(persons_file)
credits_writer = csv.writer(credits_file)
cast_writer    = csv.writer(cast_file)
crew_writer    = csv.writer(crew_file)

# (Assumes the CSVs already have header rows from the movie import.)

# Process the TV credits CSV file in chunks.
chunk_iter = pd.read_csv(
    "tv_credits.csv",
    chunksize=100000,
    sep=",",
    encoding="utf-8",
    quotechar='"'
)

chunk_count = 0
for chunk_df in chunk_iter:
    chunk_count += 1
    print(f"Processing TV chunk #{chunk_count} with {len(chunk_df)} rows...")
    # Local caches for this chunk (to minimize repeated writes within the chunk)
    local_persons_cache = set()
    local_credits_cache = set()

    for _, row in chunk_df.iterrows():
        

        content_id = row["id"]  # TV TMDB ID

        # Parse cast JSON
        cast_data = []
        if "cast" in row and pd.notna(row["cast"]):
            try:
                cast_data = json.loads(row["cast"])
            except Exception:
                pass

        # Parse crew JSON
        crew_data = []
        if "crew" in row and pd.notna(row["crew"]):
            try:
                crew_data = json.loads(row["crew"])
            except Exception:
                pass

        # Process cast members
        for member in cast_data:
            if "roles" in member and member["roles"]:
                role_info = member["roles"][0]  # Get the first role entry.
                credit_id = role_info.get("credit_id", "")
                character = role_info.get("character", "")
            else:
                continue

            person_id = member.get("id")
            if not credit_id or not person_id:
                continue

            # Debug prints to verify values:

            if person_id not in global_persons_cache:
                global_persons_cache.add(person_id)
                persons_writer.writerow([
                    person_id,
                    member.get("name", ""),
                    member.get("profile_path", "")
                ])

            if credit_id not in global_credits_cache:
                global_credits_cache.add(credit_id)
                credits_writer.writerow([credit_id, content_id, "movie"])

            # Write the CSV row using the extracted 'character'
            cast_writer.writerow([
                credit_id,
                person_id,
                character,
                member.get("order", "")
            ])


        # Process crew members
        for member in crew_data:
            if "roles" in member and member["roles"]:
                credit_id = member["roles"][0].get("credit_id")
            person_id = member.get("id")
            if not credit_id or not person_id:
                continue
            credit_id = str(credit_id)
            person_id = str(person_id)

            if person_id not in global_persons_cache and person_id not in local_persons_cache:
                local_persons_cache.add(person_id)
                persons_writer.writerow([
                    person_id,
                    member.get("name", ""),
                    member.get("profile_path", "")
                ])

            if credit_id not in global_credits_cache and credit_id not in local_credits_cache:
                local_credits_cache.add(credit_id)
                credits_writer.writerow([credit_id, content_id, "tv"])

            crew_pair = (credit_id, person_id)
            if crew_pair not in global_crew_cache:
                global_crew_cache.add(crew_pair)
                crew_writer.writerow([
                    credit_id,
                    person_id,
                    member.get("department", ""),
                    member.get("job", "")
                ])

    # Update global caches with new keys found in this chunk.
    global_persons_cache.update(local_persons_cache)
    global_credits_cache.update(local_credits_cache)

# Close all files.
persons_file.close()
credits_file.close()
cast_file.close()
crew_file.close()

print("TV credits CSV files appended with unique data.")


Loaded 0 persons, 0 credits, 0 cast entries, and 0 crew entries from existing CSVs.
Processing TV chunk #1 with 100000 rows...
Processing TV chunk #2 with 96692 rows...
TV credits CSV files appended with unique data.


In [9]:
df = pd.read_csv("tv_credits.csv")

In [10]:
df.head()

,cast,crew,id
0,"[{""adult"": false, ""gender"": 2, ""id"": 12670, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 105400, ""...",1
1,"[{""adult"": false, ""gender"": 2, ""id"": 23629, ""k...","[{""adult"": false, ""gender"": 0, ""id"": 1450356, ...",2
2,"[{""adult"": false, ""gender"": 1, ""id"": 139551, ""...",[],3
3,"[{""adult"": false, ""gender"": 1, ""id"": 34902, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 133502, ""...",4
4,"[{""adult"": false, ""gender"": 1, ""id"": 557975, ""...","[{""adult"": false, ""gender"": 1, ""id"": 1211826, ...",5


In [12]:
df2 = pd.read_csv("movies_credits.csv")
df2.head()

,cast,crew,id
0,"[{""adult"": false, ""gender"": 2, ""id"": 54768, ""k...","[{""adult"": false, ""gender"": 2, ""id"": 53836, ""k...",2
1,"[{""adult"": false, ""gender"": 2, ""id"": 4826, ""kn...","[{""adult"": false, ""gender"": 1, ""id"": 54766, ""k...",3
2,"[{""adult"": false, ""gender"": 2, ""id"": 3129, ""kn...","[{""adult"": false, ""gender"": 1, ""id"": 3110, ""kn...",5
3,"[{""adult"": false, ""gender"": 2, ""id"": 2880, ""kn...","[{""adult"": false, ""gender"": 2, ""id"": 2042, ""kn...",6
4,[],"[{""adult"": false, ""gender"": 0, ""id"": 34833, ""k...",8


In [14]:

# Disable truncation of column display
pd.set_option('display.max_colwidth', None)

# Assuming your DataFrame is named df and has an 'id' column:
row = df.loc[df['id'] == 1]

# To see the entire row:
print(row.to_string())

# Or if you only want the full content of a specific column, e.g. 'cast':
print(row['cast'].iloc[0])
